In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages
!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

right_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_right_fulltext.csv')
right_dataset = right_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
right_dataset = right_dataset.train_test_split(test_size=0.2)

# Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_right_dataset = right_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=right_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1449 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1062 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1147 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1718 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2894 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2079 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1350 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1293 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_right_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/5489 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1373 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
#finetune the pretrained model on the dataset

training_args = TrainingArguments(
    output_dir="distillgpt2_right_fulltext_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/tobijen/distillgpt2_right_fulltext_torch into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.268300,3.181575
2,3.200500,3.158486
3,3.152400,3.146352
4,3.127400,3.141380
5,3.107400,3.138345
6,3.081700,3.137761


TrainOutput(global_step=28452, training_loss=3.1641508821326267, metrics={'train_runtime': 3254.8542, 'train_samples_per_second': 69.92, 'train_steps_per_second': 8.741, 'total_flos': 7433239312465920.0, 'train_loss': 3.1641508821326267, 'epoch': 6.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 23.05


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/312M [00:00<?, ?B/s]

Upload file runs/Jul24_16-43-34_cc22a5ddb863/events.out.tfevents.1690226215.cc22a5ddb863.155.1:   0%|         …

Upload file runs/Jul24_16-43-34_cc22a5ddb863/events.out.tfevents.1690217099.cc22a5ddb863.155.0:   0%|         …

To https://huggingface.co/tobijen/distillgpt2_left_headings_torch
   b3cdcf0..9accfd7  main -> main

   b3cdcf0..9accfd7  main -> main

To https://huggingface.co/tobijen/distillgpt2_left_headings_torch
   9accfd7..f3880de  main -> main

   9accfd7..f3880de  main -> main



'https://huggingface.co/tobijen/distillgpt2_left_headings_torch/commit/9accfd702bd260ec51889b8759db25659a4e3958'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/distillgpt2_right_fulltext_torch")

# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
# generate text with the pipeline module from the transformers library
from transformers import pipeline

generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/distillgpt2_right_fulltext_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Gay marriage is now part of federal court cases in some states.\n\nThe law follows arguments by the Obama administration — that marriage can no longer be made on the birth certificate, no matter what religious convictions might be.\n\nJustice Stephen Breyer, a libertarian and Trump ally, said the Constitution protects states that have laws that prohibit federal government from recognizing their marriages.\n\nIn the case now in Delaware, the Supreme Court threw out gay marriage entirely.\n\nJustice Ruth Bader Ginsburg said in a'}]
social_2  =>  Abortion is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Abortion is one of the many high-profile cases of rape. The first case of rape in this country came to New York City during the 1970s, when 19-year-old Laura Davis was attacked by a gang of gang members at a party. She was later raped and later taken to a psychiatric ward.\n\nThe court heard during the trial the victim told the jurors “I had been raped and then I woke up and thought I was being raped.”\n\n“I could'}]
education_1  =>  Public education is


...

[{'generated_text': 'Charter schools are getting the biggest increases in students from the lower grades.\n\n“The reason has always been that we’re always looking towards those grades. It’s one thing to focus on how we get our kids into schools,” said Scott Dolan, president and CEO of the Alliance on Public Education’s Board of Trustees Association, a leading advocacy group. “So that, frankly, schools can benefit the most.\n\n“But that’s'}]
economic_1  =>  The increase of taxes is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "The increase of taxes is part of a renewed campaign against Trump.\n\nHowever, the U.S. has fallen behind a dozen countries as both have a long history of taxing their citizens for their specific activity.\n\nWith China, meanwhile, increasingly imposing tariffs on U.S. firms and businesses, officials have been forced to cut back on investments and hiring managers.\n\nChina's Ministry of Trade and Industry said in a statement that the tariffs would stop the sale of luxury goods, luxury goods and other items."}]
economic_2  =>  Government regulations on businesses are


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Government regulations on businesses are subject to the “regulatory mischief theory” from the Department of Commerce at the behest of the tobacco industry.\n\nIn response, the administration announced it would remove the regulations from the website and ban all tobacco products made with the EPA from the website.\n\nThe Biden administration’s move follows a yearlong effort to dismantle the so-called “tobacco tax” imposed by the Obama-era “backslash” of high-fructose corn'}]


In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is now part of federal court cases in some states.\n\nThe law follows arguments by the Obama administration — that marriage can no longer be made on the birth certificate, no matter what religious convictions might be.\n\nJustice Stephen Breyer, a libertarian and Trump ally, said the Constitution protects states that have laws that prohibit federal government from recognizing their marriages.\n\nIn the case now in Delaware, the Supreme Court threw out gay marriage entirely.\n\nJustice Ruth Bader Ginsburg said in a',
 'social_2': 'Abortion is one of the many high-profile cases of rape. The first case of rape in this country came to New York City during the 1970s, when 19-year-old Laura Davis was attacked by a gang of gang members at a party. She was later raped and later taken to a psychiatric ward.\n\nThe court heard during the trial the victim told the jurors “I had been raped and then I woke up and thought I was being raped.”\n\n“I could',
 'education_1': '

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "distilgpt2_right_fulltext": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)